In [1]:
import re
import pandas as pd
import spacy
from transformers import FlaubertTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def extract_articles_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Pattern pour identifier les codes d'article
    mixed_code_pattern = re.compile(r'\b(?=[A-Z0-9]{10}\b)(?=[A-Z]*[0-9][A-Z]*)(?=[0-9]*[A-Z][0-9]*)[A-Z0-9]{10}')

    # Recherche de tous les codes correspondant à ce pattern dans le contenu
    mixed_found_codes = mixed_code_pattern.findall(content)

    # Compter les codes uniques correspondant aux nouveaux critères
    unique_mixed_found_codes = set(mixed_found_codes)
    number_of_unique_mixed_found_articles = len(unique_mixed_found_codes)

    # Détermination des limites des articles en fonction des codes uniques identifiés
    article_boundaries = list(mixed_code_pattern.finditer(content))

    # Extraction de chaque article avec son code
    articles = []
    for i in range(len(article_boundaries)):
        start_index = article_boundaries[i].start()
        end_index = article_boundaries[i+1].start() if i+1 < len(article_boundaries) else None
        article_code = content[start_index:article_boundaries[i].end()].strip()
        article_text = content[article_boundaries[i].end():end_index].strip()
        articles.append((article_code, article_text))

    # Création d'un DataFrame pour stocker chaque article avec son code
    articles_df = pd.DataFrame(articles, columns=['Code', 'Text'])

    return articles_df

In [3]:
file_path = r"C:\Users\33672\OneDrive\Bureau\TER Docs\corpus_FRESA - Texte - Pertinence\Entrainement - txt\Pertinent.txt"
articles_df = extract_articles_from_file(file_path)

,Pertinence,Code,Text
0,1,2G9VHL0NWT,",Adaptation climatique: Un appel à privilégie..."
1,1,2Z6BFRQE9N,",""Lutte contre la désertification et la dégrad..."
2,1,DO5FX9BGHL,",""------------------------- sur la production ..."
3,1,NZI7QXUVNT,",Sûreté et sécurité au port de Cotonoun : Béni..."
4,1,RZDF3OO2DN,",Production de maïs : les jeunes s'y mettent a..."


In [ ]:

##UNIQUEMENT POUR DATASET ENTRAINEMENT

# Ajouter une nouvelle colonne contenant des '1 ou 0' au début du DataFrame pour déterminer la pertinence ou non de l'article
articles_df.insert(0, 'Pertinence', 1)

In [ ]:
articles_df.head()  

In [4]:
##Tokenisation

In [5]:
# Chargez le tokenizer pour FlauBERT
flaubert_tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased')

def tokenize_articles(articles_df):
    # Appliquer la tokenisation à chaque article
    tokenized_articles = []
    for _, row in articles_df.iterrows():
        article_text = row['Text']
        tokens = flaubert_tokenizer.tokenize(article_text)
        tokenized_articles.append(tokens)
    
    return tokenized_articles

tokenized_articles = tokenize_articles(articles_df)

# Affiche les premiers articles tokenisés 
print(tokenized_articles[:5])


[[',</w>', 'Adap', 'tation</w>', 'climatique</w>', ':</w>', 'Un</w>', 'appel</w>', 'à</w>', 'privilégier</w>', "l'</w>", 'agro', 'écologie</w>', '"</w>', ',</w>', 'Adap', 'tation</w>', 'climatique</w>', ':</w>', 'Un</w>', 'appel</w>', 'à</w>', 'privilégier</w>', "l'</w>", 'agro', 'éc', 'ologi', 'e', 'Trois</w>', 'actions</w>', 'phares</w>', 'pour</w>', 'placer</w>', "l'</w>", 'agro', 'écologie</w>', 'au</w>', 'ceur</w>', 'de</w>', "l'</w>", 'adaptation</w>', 'climatique</w>', '.</w>', "C'</w>", 'est</w>', 'ce</w>', 'que</w>', 'proposent</w>', 'des</w>', 'acteurs</w>', 'de</w>', "l'</w>", 'Alliance</w>', 'pour</w>', 'la</w>', 'souveraineté</w>', 'alimentaire</w>', 'en</w>', 'Afrique</w>', '(</w>', 'Af', 'sa</w>', ')</w>', 'à</w>', 'la</w>', 'suite</w>', 'des</w>', 'assises</w>', 'tenues</w>', 'à</w>', 'Ad', 'dis-', 'Abeba</w>', 'du</w>', '19</w>', 'au</w>', '21</w>', 'septembre</w>', '20', '22.', "L'</w>", 'Afrique</w>', 'peut</w>', 'conduire</w>', 'le</w>', 'monde</w>', 'dans</w>', 'la

In [6]:
##Lemmatization

In [7]:
# Chargement de SpaCy
nlp = spacy.load('fr_core_news_sm')

def lemmatize_articles(articles_df):
    lemmatized_articles = []
    for _, row in articles_df.iterrows():
        article_text = row['Text']
        doc = nlp(article_text)
        lemmas = [token.lemma_ for token in doc]
        lemmatized_articles.append(' '.join(lemmas))
    
    return lemmatized_articles

lemmatized_articles = lemmatize_articles(articles_df)

print(lemmatized_articles[:2])


[', adaptation climatique : un appel à \xa0  privilégier le agroécologie,"adaptation climatique : un appel à \xa0  privilégier le agroécologie \n Trois action phare pour placer le agroécologie au ceur de le adaptation climatique . ce être ce que proposer de acteur de le Alliance pour le souveraineté alimentaire en Afrique ( Afsa ) à \xa0  le suite de assise tenu à \xa0  Addis - Abeba de 19 au 21 septembre 2022 . \n le Afrique pouvoir conduire le monde dans le transition vers un système alimentaire durable grce à \xa0  le agroécologie . ce être le leitmotiv de petit exploitant agricole , un jeune , un femme , un chercheur et scientifique , un écologiste , faire partie de le Alliance pour le souveraineté alimentaire en Afrique ( Afsa ) . il se agir de plus grand mouvement de le société civil de continent représentant 200 million de paysan , pêcheur , éleveur , consommateur , groupe confessionnel et peuple autochtone . réuni à \xa0  Addis - Abeba en ethiopie de 19 au 21 septembre dernier 

In [8]:
##Stopword Removal

In [9]:
nlp = spacy.load('fr_core_news_sm')

def remove_stopwords(articles_df):
    non_stop_articles = []
    for _, row in articles_df.iterrows():
        article_text = row['Text']
        doc = nlp(article_text)
        filtered_tokens = [token.text for token in doc if not token.is_stop]
        filtered_article = ' '.join(filtered_tokens)
        non_stop_articles.append(filtered_article)
    
    return non_stop_articles

non_stop_articles = remove_stopwords(articles_df)

print(non_stop_articles[:2])


[', Adaptation climatique : appel \xa0  privilégier agroécologie,"Adaptation climatique : appel \xa0  privilégier agroécologie \n actions phares placer agroécologie ceur adaptation climatique . proposent acteurs Alliance souveraineté alimentaire Afrique ( Afsa ) \xa0  suite assises tenues \xa0  Addis - Abeba 19 21 septembre 2022 . \n Afrique conduire monde transition systèmes alimentaires durables grce \xa0  agroécologie . leitmotiv petits exploitants agricoles , jeunes , femmes , chercheurs scientifiques , écologistes , partie Alliance souveraineté alimentaire Afrique ( Afsa ) . agit grand mouvement société civile continent représentant 200 millions paysans , pêcheurs , éleveurs , consommateurs , groupes confessionnels peuples autochtones . Réunis \xa0  Addis - Abeba Ethiopie 19 21 septembre dernier établir feuille route Afrique adaptation agroécologie , lancent appel \xa0  action \xa0  Cop 27 au-delà \xa0 . « demandons Cop 27 place agroécologie centre adaptation climatique Afrique , 

In [10]:
##Vectorisation (TF-IDF)

In [11]:
def vectorize_articles(articles_df):
    vectorizer = TfidfVectorizer()

    tfidf_matrix = vectorizer.fit_transform(articles_df['Text'])

    return tfidf_matrix, vectorizer.get_feature_names_out()

tfidf_matrix, feature_names = vectorize_articles(articles_df)

tfidf_matrix.shape, feature_names[:10]


((11, 2534),
 array(['00', '000', '013', '02', '03', '04', '05', '06', '07', '09'],
       dtype=object))

In [ ]:
# Converti la matrice TF-IDF en DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

In [ ]:

##UNIQUEMENT POUR DATASET ENTRAINEMENT

##re ajout de la pertinence

In [ ]:
# Ajouter la colonne de pertinence au DataFrame
combined_df = pd.concat([articles_df['Pertinence'], tfidf_df], axis=1)

In [ ]:
##enregistrement du dataset

In [ ]:
# Extraire le nom de base du fichier
base_name = os.path.basename(file_path)

# Retirer l'extension du fichier et ajouter '_traité'
new_file_name = os.path.splitext(base_name)[0] + '_traité.pkl'

# Définir le chemin du nouveau fichier pickle
new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)

# Sauvegarder le DataFrame au format pickle
combined_df.to_pickle(new_file_path)

print(f"DataFrame sauvegardé au chemin : {new_file_path}")

In [ ]:

##UNIQUEMENT POUR DATASET ENTRAINEMENT

##Concaténation dataset pertinent et non pertinent

# Chemin du dossier courant où se trouve le script Python
current_folder = os.path.dirname(os.path.abspath(__file__))

# Noms des fichiers pickle pour les datasets pertinents et non pertinents
pertinent_file_name = 'Pertinent_traité.pkl'
non_pertinent_file_name = 'Non_Pertinent_traité.pkl'

# Chemins complets des fichiers pickle
pertinent_file_path = os.path.join(current_folder, pertinent_file_name)
non_pertinent_file_path = os.path.join(current_folder, non_pertinent_file_name)

# Charger les DataFrames
pertinent_df = pd.read_pickle(pertinent_file_path)
non_pertinent_df = pd.read_pickle(non_pertinent_file_path)

# Concaténer les DataFrames
combined_df = pd.concat([pertinent_df, non_pertinent_df], ignore_index=True)

# Afficher les premières lignes du DataFrame combiné pour vérification
print(combined_df.head())